<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/Dataset_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os import listdir
from scipy.io import loadmat
filename = []
for mat_file in listdir('/content/drive/MyDrive/sound2meg/mat_files'):
  filename.append(mat_file)
sizes = []
for file in filename:
  mat_file = loadmat('/content/drive/MyDrive/sound2meg/mat_files/' + file)  #Was really slow to import every file to check the size, hence not included in the class
  Signal = mat_file['data']
  sizes.append(Signal.shape[2])

In [ ]:
from scipy.io import savemat
file_sizes = {'file_sizes': sizes}
savemat('file_sizes.mat', file_sizes)

In [ ]:
np.array(loadmat('/content/drive/MyDrive/file_sizes.mat')['file_sizes'][0])

array([147, 168, 171, 166, 167, 178, 170, 152, 180,  99, 165, 169, 170,
       169, 170, 171, 164, 174, 160, 174, 183, 171, 178, 162, 155, 167,
       163, 154, 164,  90, 174, 168, 162, 173, 163, 170, 170, 169, 170,
       178, 174, 170, 163, 162, 164, 170, 179, 164, 178, 146, 177, 170,
       160, 169, 178, 153, 166, 172, 158, 167, 164, 105, 174, 169, 169,
       166,  49, 172, 168, 164, 166, 166, 165, 177, 161, 166, 169, 168,
       178, 165, 168, 160, 171, 171, 169, 165, 173, 157, 170, 171, 157,
       171, 171, 172, 156, 175, 158, 168, 168, 164])

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from scipy.io import loadmat
from os import listdir
import numpy as np
from sklearn.preprocessing import RobustScaler
import random

class Sound2MEGDataset(Dataset):
  def __init__(self, meg_files_path):
    #self.wav_files = wav_files
    self.meg_files_path = meg_files_path
    self.sizes = np.array(loadmat('/content/drive/MyDrive/file_sizes.mat')['file_sizes'][0])
    self.filename = []
    for mat_file in listdir(self.meg_files_path):
      self.filename.append(mat_file)
  def __len__(self):
    return sum(self.sizes)
  def __getitem__(self, idx):
    for i in range(len(self.sizes)):
      if np.cumsum(self.sizes)[i] > idx:
        idx_file = i
        break
    mat_file = loadmat(self.meg_files_path + self.filename[idx_file])
    if idx_file == 0:
      idx_sound = idx
    else:
      idx_sound = idx - np.cumsum(self.sizes)[idx_file - 1]
    audio_file = mat_file['audiofiles'][0, idx_sound][0][0:3]
    Sound_Signal = np.load('/content/drive/MyDrive/Mel_Embedding/mel_' + audio_file + '.npy')
    mat_file = mat_file['data']
    MEG_Signal = np.float32(mat_file[:273, :, idx_sound])
    mean_5 = np.mean(MEG_Signal[:,:60], axis=1)
    MEG_Signal = MEG_Signal - mean_5[:,None]
    scaler = RobustScaler().fit(MEG_Signal)
    MEG_Signal = scaler.transform(MEG_Signal)
    return torch.from_numpy(MEG_Signal), torch.from_numpy(Sound_Signal) , idx_file

Dataset = Sound2MEGDataset('/content/drive/MyDrive/sound2meg/mat_files/')  #Can make a vector of sizes in the directory and import it from there
training_data, validation_data, test_data = random_split(Dataset, [0.7, 0.2, 0.1], generator=torch.Generator().manual_seed(42))
Training_Data_Batches = DataLoader(training_data, batch_size = 128, shuffle = True)
#for Sample, Subject in Training_Data:
#  print(Sample.shape)
Sample, Sound, Subject = Dataset[14462]

#Random Batch
#batch_size = 3
#Samples = torch.zeros(batch_size, 273, 360)
#j = 0
#Subjects = []
#for i in random.sample(range(0, 16446), batch_size):
#  S, Sub = Dataset[i]
#  Samples[j] = S[:273,:]
#  j = j + 1
#  Subjects.append(Sub)

#print(Samples.shape)
#print(Subjects)

In [ ]:
from spatial_attention import Net as BrainModule
Model = BrainModule()
X = Model(Samples, Subjects)
print(X.shape)

torch.Size([3, 270, 360, 1])
torch.Size([3, 120, 360, 1])


In [ ]:
print(np.cumsum(np.array(loadmat('/content/drive/MyDrive/file_sizes.mat')['file_sizes'][0])))

169


In [ ]:
print(sizes)

[147, 168, 171, 166, 167, 178, 170, 152, 180, 99, 165, 169, 170, 169, 170, 171, 164, 174, 160, 174, 183, 171, 178, 162, 155, 167, 163, 154, 164, 90, 174, 168, 162, 173, 163, 170, 170, 169, 170, 178, 174, 170, 163, 162, 164, 170, 179, 164, 178, 146, 22, 177, 170, 160, 169, 178, 153, 166, 172, 158, 167, 164, 105, 0, 174, 169, 169, 166, 49, 172, 168, 164, 166, 166, 165, 177, 161, 166, 169, 168, 178, 165, 168, 160, 171, 171, 169, 165, 173, 157, 170, 171, 157, 171, 171, 172, 156, 175, 158, 168, 168, 164]


In [ ]:
mat_file = loadmat('/content/drive/MyDrive/sound2meg/mat_files/' + filename[66])['data']
print(mat_file.shape[2])
MEG_Signal = np.float32(mat_file[:273, :, 163])

49


IndexError: ignored